In [1]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
#!pip install python-gdcm
#!pip install pylibjpeg
#!pip install pylibjpeg-libjpeg==2.2.0
#!pip install pylibjpeg-openjpeg==2.3.0
#!pip install matplotlib==3.10.3
#!pip install scikit-learn==1.7.0
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
#!pip install pydicom

#!pip install tensorflow-io==0.37.1
!pip install tensorflow-io==0.37.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow_io

#!pip install tensorflow==2.16.1
!pip install tensorflow==2.16.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow
# Don't need it while inference
#!pip install tensorflow-tpu==2.16.1 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Looking in links: /kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow_io
Looking in links: /kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow
Processing /kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow/tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow/ml_dtypes-0.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from tensorflow==2.16.1)
Processing /kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow/tensorboard-2.16.2-py3-none-any.whl (from tensorflow==2.16.1)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0

In [ ]:
"""!pip download tensorflow-io==0.37.1 -d /kaggle/working/mysitepackages/tensorflow_io
!pip download tensorflow==2.16.1 -d /kaggle/working/mysitepackages/tensorflow"""

In [ ]:
"""from zipfile import ZipFile

dirName = '/kaggle/working/mysitepackages'
zipName = 'tensorflow_working_combo.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [3]:
"""import pydicom
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import log_loss
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

In [4]:
print(tf.__version__)
print(tfio.__version__)

2.16.1
0.37.1


In [5]:
model = keras.models.\
load_model("/kaggle/input/rsna-aneurysm-cta-base/tensorflow2/inception-v3-transfer-learning/1/rsna_aneurysm_keras_base.h5")

In [6]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 4


In [7]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

In [8]:
def process_cta(image: np.ndarray, dcm):
    """
    Specific preprocessing for CTA
    - Window/level optimization for vessel visualization
    - Vessel enhancement
    """
    # Get window settings (typical for CTA)
    window_center = 100  # Typical for CTA
    window_width = 700   # Typical for CTA
    
    # Override with DICOM values if available
    if hasattr(dcm, 'WindowCenter') and hasattr(dcm, 'WindowWidth'):
        window_center = dcm.WindowCenter
        window_width = dcm.WindowWidth
        
        # Handle multiple window settings
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = window_center[0]
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = window_width[0]
    
    # Apply window/level
    min_value = window_center - window_width // 2
    max_value = window_center + window_width // 2
    image = np.clip(image, min_value, max_value)
    
    # Normalize to [0,1]
    image = (image - min_value) / (max_value - min_value + 1e-10)
    
    return image

def read_and_parse_dicom_files_tensorflow_infer(dcm_path, f_coord=None):
    tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    image = process_cta(image, dcm_path)
    #if f_coord:
    specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]]
    )
    # Squeeze to remove the extra dimension of size 1
    specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    """else:
        specific_slice_tensor = tf.squeeze(image)"""
    #print("specific_slice_tensor - ", specific_slice_tensor.shape)
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    #print("sqzd_image -", sqzd_image.shape)
    return sqzd_image

def load_dicom_tf_cta(filepath_tensor, f_coord_tensor=None):
    """
    Wrapper function to call read_and_parse_dicom_files_tensorflow_train within a tf.data.Dataset pipeline.
    """
    #if f_coord_tensor:
    image = tf.py_function(
        read_and_parse_dicom_files_tensorflow_train_cta,
        inp=[filepath_tensor, f_coord_tensor],
        Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )
    """else:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )"""
    # Ensure the shape is defined if known, as tf.py_function can lose shape info
    #image.set_shape([128, 128]) # Example for a 2D image, adjust as needed
    return image

In [9]:
pos_dict = {0: 'anterior_communicating_artery',
 1: 'basilar_tip',
 2: 'left_anterior_cerebral_artery',
 3: 'left_infraclinoid_internal_carotid_artery',
 4: 'left_middle_cerebral_artery',
 5: 'left_posterior_communicating_artery',
 6: 'left_supraclinoid_internal_carotid_artery',
 7: 'aneurysm_present',
 8: 'other_posterior_circulation',
 9: 'right_anterior_cerebral_artery',
 10: 'right_infraclinoid_internal_carotid_artery',
 11: 'right_middle_cerebral_artery',
 12: 'right_posterior_communicating_artery',
 13: 'right_supraclinoid_internal_carotid_artery'}

SUBMISSION_COLS = [
    'SeriesInstanceUID',
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

def format_column_name(snake_str):
    """Convert snake_case string to Title Case with spaces"""
    # Split the string into words and capitalize each word
    return ' '.join(word.capitalize() for word in snake_str.split('_'))

def create_prediction_dataframe(predictions, label_dict, dicom_path):
    # Create a copy of the predictions to avoid modifying the original
    pred_copy = predictions.copy()
    
    # Subtract the 8th element (index 7) from 1
    pred_copy[0, 7] = 1 - pred_copy[0, 7]

    # Start with the DICOM path
    data_dict = {'DICOM Path': dicom_path.split('/')[-2]}
    
    # Convert predictions to a dictionary where keys are labels and values are predictions
    data_dict.update({format_column_name(label_dict[i]): float(pred) for i, pred in enumerate(pred_copy[0])})
    
    # Create Polars DataFrame from the dictionary
    df = pl.DataFrame([data_dict])
    
    df.columns = SUBMISSION_COLS

    df = df.group_by("SeriesInstanceUID").agg(
        pl.col(LABEL_COLS[0]).max().alias(LABEL_COLS[0]),
        pl.col(LABEL_COLS[1]).max().alias(LABEL_COLS[1]),
        pl.col(LABEL_COLS[2]).max().alias(LABEL_COLS[2]),
        pl.col(LABEL_COLS[3]).max().alias(LABEL_COLS[3]),
        pl.col(LABEL_COLS[4]).max().alias(LABEL_COLS[4]),
        pl.col(LABEL_COLS[5]).max().alias(LABEL_COLS[5]),
        pl.col(LABEL_COLS[6]).max().alias(LABEL_COLS[6]),
        pl.col(LABEL_COLS[7]).max().alias(LABEL_COLS[7]),
        pl.col(LABEL_COLS[8]).max().alias(LABEL_COLS[8]),
        pl.col(LABEL_COLS[9]).max().alias(LABEL_COLS[9]),
        pl.col(LABEL_COLS[10]).max().alias(LABEL_COLS[10]),
        pl.col(LABEL_COLS[11]).max().alias(LABEL_COLS[11]),
        pl.col(LABEL_COLS[12]).max().alias(LABEL_COLS[12]),
        pl.col(LABEL_COLS[13]).max().alias(LABEL_COLS[13])
    )
    
    return df

In [10]:
def predict(series_path: str) -> pl.DataFrame | pd.DataFrame:
    print("Starting with Series -> ",series_path)
    all_filepaths = []
    for root, _, files in os.walk(series_path):
        for file in files:
            if file.endswith('.dcm'):
                all_filepaths.append(os.path.join(root, file))
    all_filepaths.sort()
    
    all_predictions = []
    total_files = len(all_filepaths)
    
    for dicom_path in all_filepaths:
        test_image = read_and_parse_dicom_files_tensorflow_infer(dicom_path, 0)
        test_image_for_inf = np.expand_dims(test_image, axis=0)
        prediction = model.predict(test_image_for_inf, verbose=False)
    
        pred_copy = prediction.copy()
        
        # Subtract the 8th element (index 7) from 1
        pred_copy[0, 7] = 1 - pred_copy[0, 7]
    
        # Start with the DICOM path
        data_dict = {'DICOM Path': dicom_path.split('/')[-2]}
        
        # Convert predictions to a dictionary where keys are labels and values are predictions
        data_dict.update({format_column_name(pos_dict[i]): float(pred) for i, pred in enumerate(pred_copy[0])})
    
        all_predictions.append(data_dict)
    
    final_df = pl.DataFrame(all_predictions)
    
    final_df.columns = SUBMISSION_COLS
    
    final_df = final_df.group_by("SeriesInstanceUID").agg(
        pl.col(LABEL_COLS[0]).max().alias(LABEL_COLS[0]),
        pl.col(LABEL_COLS[1]).max().alias(LABEL_COLS[1]),
        pl.col(LABEL_COLS[2]).max().alias(LABEL_COLS[2]),
        pl.col(LABEL_COLS[3]).max().alias(LABEL_COLS[3]),
        pl.col(LABEL_COLS[4]).max().alias(LABEL_COLS[4]),
        pl.col(LABEL_COLS[5]).max().alias(LABEL_COLS[5]),
        pl.col(LABEL_COLS[6]).max().alias(LABEL_COLS[6]),
        pl.col(LABEL_COLS[7]).max().alias(LABEL_COLS[7]),
        pl.col(LABEL_COLS[8]).max().alias(LABEL_COLS[8]),
        pl.col(LABEL_COLS[9]).max().alias(LABEL_COLS[9]),
        pl.col(LABEL_COLS[10]).max().alias(LABEL_COLS[10]),
        pl.col(LABEL_COLS[11]).max().alias(LABEL_COLS[11]),
        pl.col(LABEL_COLS[12]).max().alias(LABEL_COLS[12]),
        pl.col(LABEL_COLS[13]).max().alias(LABEL_COLS[13])
    )

    shutil.rmtree('/kaggle/shared', ignore_errors=True)

    return final_df.drop('SeriesInstanceUID')

In [11]:
import kaggle_evaluation.rsna_inference_server
import shutil

shutil.rmtree('/kaggle/shared', ignore_errors=True)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("inside if")
    inference_server.serve()
else:
    print("inside else")
    inference_server.run_local_gateway()
    display(pl.read_parquet('/kaggle/working/submission.parquet'))

inside else
Starting with Series ->  /kaggle/shared/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation/series/1.2.826.0.1.3680043.8.498.10057359944432090722321480667591403108


W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length av

KeyboardInterrupt: 

W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
I0000 00:00:1759329046.727323     114 chttp2_transport.cc:1287] ipv4:127.0.0.1:50051: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {grpc_status:14, http2_error:2}
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Length available in Meta Information Header
W: DcmMetaInfo: No Group Le